<a href="https://colab.research.google.com/github/natalykur/prod_like_rag/blob/main/Rag_with_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Complete RAG System for Pandas Data

# ===============================================
# SECTION 1: INSTALLATION AND SETUP
# ===============================================

# Install required packages
!pip install -q sentence-transformers
!pip install -q chromadb
!pip install -q openai
!pip install -q ragas
!pip install -q datasets
!pip install -q langchain
!pip install -q langchain-openai
!pip install -q pandas numpy scikit-learn
!pip install -q plotly

# Import libraries
import pandas as pd
import numpy as np
import json
import os
from typing import List, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

# Vector and embedding libraries
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions

# LLM and RAG libraries
import openai
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document

# Evaluation libraries
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
    answer_similarity
)
from datasets import Dataset

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Utility libraries
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
import time

print("All libraries installed and imported successfully!")


In [ ]:

# ===============================================
# SECTION 2: CONFIGURATION AND SETUP
# ===============================================
from google.colab import userdata

# Configuration
class RAGConfig:
    def __init__(self):
        self.embedding_model = 'all-MiniLM-L6-v2'
        self.chunk_size = 512
        self.chunk_overlap = 50
        self.top_k_retrieval = 5
        self.temperature = 0.3
        self.max_tokens = 500


        # OpenAI API key (set your key here)
        #self.openai_api_key = "your-openai-api-key-here"  # Replace with your actual key
        # Fetch the API key from Colab secrets
        try:
            self.openai_api_key = userdata.get('OPENAI_API_KEY')
        except userdata.SecretNotFoundError:
            print("OpenAI API key not found in Colab secrets. Please add it as 'OPENAI_API_KEY'.")
            self.openai_api_key = None # Or raise an error

        # Vector database settings
        self.collection_name = "ecommerce_products"
        self.persist_directory = "/content/chroma_db"

config = RAGConfig()

# Set OpenAI API key
os.environ["OPENAI_API_KEY"] = config.openai_api_key
openai.api_key = config.openai_api_key


In [ ]:

# ===============================================
# SECTION 3: DATA GENERATION
# ===============================================

def generate_ecommerce_data(num_records=500):
    """Generate sample e-commerce dataset"""
    import random

    np.random.seed(42)
    random.seed(42)

    categories = ['Electronics', 'Clothing', 'Home & Garden', 'Books', 'Sports', 'Beauty']

    products_by_category = {
        'Electronics': ['iPhone 14', 'Samsung Galaxy S23', 'MacBook Pro', 'Dell XPS', 'iPad Air', 'Sony Headphones'],
        'Clothing': ['Nike T-Shirt', 'Levis Jeans', 'Adidas Sneakers', 'H&M Dress', 'Zara Jacket', 'Uniqlo Hoodie'],
        'Home & Garden': ['Coffee Maker', 'Vacuum Cleaner', 'Garden Hose', 'Dining Table', 'Bed Sheets', 'Plant Pot'],
        'Books': ['Python Programming', 'Data Science Handbook', 'Machine Learning', 'Deep Learning', 'Statistics', 'AI Ethics'],
        'Sports': ['Yoga Mat', 'Tennis Racket', 'Running Shoes', 'Basketball', 'Dumbbells', 'Bicycle'],
        'Beauty': ['Face Cream', 'Shampoo', 'Lipstick', 'Perfume', 'Sunscreen', 'Hair Oil']
    }

    price_ranges = {
        'Electronics': (50, 2000),
        'Clothing': (15, 200),
        'Home & Garden': (20, 500),
        'Books': (10, 60),
        'Sports': (25, 300),
        'Beauty': (8, 100)
    }

    data = []

    for i in range(num_records):
        category = random.choice(categories)
        product = random.choice(products_by_category[category])

        min_price, max_price = price_ranges[category]
        price = round(random.uniform(min_price, max_price), 2)
        rating = round(np.random.beta(7, 2) * 5, 1)

        base_sales = random.randint(50, 1000)
        price_factor = 1 - (price - min_price) / (max_price - min_price) * 0.3
        sales = int(base_sales * price_factor)

        revenue = price * sales
        discount = random.choice([0, 5, 10, 15, 20, 25]) if random.random() < 0.3 else 0
        stock_level = random.randint(0, 500)
        stock_status = 'Out of Stock' if stock_level == 0 else 'Low Stock' if stock_level < 50 else 'In Stock'

        satisfaction = 'High' if rating >= 4.0 else 'Medium' if rating >= 3.0 else 'Low'

        data.append({
            'product_id': f'P{i+1:04d}',
            'product_name': product,
            'category': category,
            'price': price,
            'rating': rating,
            'sales_quantity': sales,
            'revenue': round(revenue, 2),
            'discount_percent': discount,
            'stock_level': stock_level,
            'stock_status': stock_status,
            'customer_satisfaction': satisfaction
        })

    return pd.DataFrame(data)

# Generate the dataset
print("Generating e-commerce dataset...")
df = generate_ecommerce_data(500)
print(f"Dataset generated with {len(df)} records")
print("\nDataset sample:")
print(df.head())

# Save dataset
df.to_csv('/content/ecommerce_products.csv', index=False)
print("\nDataset saved to /content/ecommerce_products.csv")


In [ ]:
# ===============================================
# SECTION 4: TEXT PROCESSING AND VECTORIZATION
# ===============================================

# **Text:** Processing the data and creating vector embeddings.

# **Paragraph:** This section defines a `DataProcessor` class responsible for transforming the Pandas DataFrame into text chunks and generating vector embeddings for these chunks. It uses the `SentenceTransformer` library to create embeddings based on the configured model.

class DataProcessor:
    def __init__(self, config: RAGConfig):
        self.config = config
        self.embedding_model = SentenceTransformer(config.embedding_model)

    def create_document_chunks(self, df: pd.DataFrame) -> List[Dict]:
        """Convert DataFrame rows to text chunks for vectorization"""
        chunks = []

        for idx, row in df.iterrows():
            # Create comprehensive text representation
            text = f"Product: {row['product_name']}\n"
            text += f"Category: {row['category']}\n"
            text += f"Price: ${row['price']}\n"
            text += f"Rating: {row['rating']}/5\n"
            text += f"Sales Quantity: {row['sales_quantity']} units\n"
            text += f"Revenue: ${row['revenue']:,.2f}\n"
            text += f"Discount: {row['discount_percent']}%\n"
            text += f"Stock Level: {row['stock_level']} units\n"
            text += f"Stock Status: {row['stock_status']}\n"
            text += f"Customer Satisfaction: {row['customer_satisfaction']}\n"

            # Add contextual information
            text += f"\nThis is a {row['category'].lower()} product with "
            text += f"{'high' if row['rating'] >= 4.0 else 'medium' if row['rating'] >= 3.0 else 'low'} customer rating. "

            if row['discount_percent'] > 0:
                text += f"Currently on sale with {row['discount_percent']}% discount. "

            text += f"Stock status is {row['stock_status'].lower()}."

            chunks.append({
                'id': row['product_id'],
                'text': text,
                'metadata': {
                    'product_id': row['product_id'],
                    'product_name': row['product_name'],
                    'category': row['category'],
                    'price': row['price'],
                    'rating': row['rating'],
                    'sales_quantity': row['sales_quantity'],
                    'revenue': row['revenue'],
                    'stock_status': row['stock_status']
                }
            })

        return chunks

    def create_embeddings(self, chunks: List[Dict]) -> List[np.ndarray]:
        """Create embeddings for text chunks"""
        texts = [chunk['text'] for chunk in chunks]
        embeddings = self.embedding_model.encode(texts, show_progress_bar=True)
        return embeddings

# Initialize processor
processor = DataProcessor(config)

# Create chunks and embeddings
print("Processing data into chunks...")
chunks = processor.create_document_chunks(df)
print(f"Created {len(chunks)} chunks")

print("\nSample chunk:")
print(chunks[0]['text'][:300] + "...")

print("\nCreating embeddings...")
embeddings = processor.create_embeddings(chunks)
print(f"Created embeddings with shape: {embeddings.shape}")

In [ ]:
# ===============================================
# SECTION 5: VECTOR DATABASE SETUP
# ===============================================

# **Text:** Setting up the vector database.

# **Paragraph:** This section configures and initializes a ChromaDB vector database to store the embeddings of the product information. It defines a `VectorDatabase` class that handles the creation of a collection, adding documents (chunks and metadata), and searching for relevant information based on a query.

class VectorDatabase:
    def __init__(self, config: RAGConfig):
        self.config = config
        self.client = chromadb.PersistentClient(path=config.persist_directory)
        self.embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
            model_name=config.embedding_model
        )
        self.collection = None

    def create_collection(self, chunks: List[Dict]):
        """Create and populate vector database collection"""
        try:
            # Delete existing collection if it exists
            self.client.delete_collection(name=self.config.collection_name)
        except:
            pass

        # Create new collection
        self.collection = self.client.create_collection(
            name=self.config.collection_name,
            embedding_function=self.embedding_function
        )

        # Add documents to collection
        ids = [chunk['id'] for chunk in chunks]
        documents = [chunk['text'] for chunk in chunks]
        metadatas = [chunk['metadata'] for chunk in chunks]

        self.collection.add(
            ids=ids,
            documents=documents,
            metadatas=metadatas
        )

        print(f"Added {len(chunks)} documents to vector database")

    def search(self, query: str, top_k: int = 5) -> Dict:
        """Search for relevant documents"""
        if not self.collection:
            self.collection = self.client.get_collection(name=self.config.collection_name)

        results = self.collection.query(
            query_texts=[query],
            n_results=top_k
        )

        return {
            'documents': results['documents'][0],
            'metadatas': results['metadatas'][0],
            'distances': results['distances'][0],
            'ids': results['ids'][0]
        }

# Initialize vector database
print("Setting up vector database...")
vector_db = VectorDatabase(config)
vector_db.create_collection(chunks)
print("Vector database setup complete!")

# Test search
test_query = "What are the highest rated electronics?"
search_results = vector_db.search(test_query, top_k=3)
print(f"\nTest search for: '{test_query}'")
print(f"Found {len(search_results['documents'])} relevant documents")

In [ ]:
# ===============================================
# SECTION 6: RAG SYSTEM IMPLEMENTATION
# ===============================================

# **Text:** Implementing the Retrieval-Augmented Generation (RAG) system.

# **Paragraph:** This section defines the core `RAGSystem` class. It integrates the vector database for retrieval and a language model (GPT-3.5-turbo) for generating answers. The system takes a user query, retrieves relevant product information from the vector database, and then uses the LLM to generate a comprehensive answer based on the retrieved context.

class RAGSystem:
    def __init__(self, config: RAGConfig, vector_db: VectorDatabase):
        self.config = config
        self.vector_db = vector_db
        self.llm = ChatOpenAI(
            model_name="gpt-3.5-turbo",
            temperature=config.temperature,
            max_tokens=config.max_tokens
        )

    def retrieve(self, query: str) -> List[str]:
        """Retrieve relevant documents for a query"""
        search_results = self.vector_db.search(query, top_k=self.config.top_k_retrieval)
        return search_results['documents']

    def generate_answer(self, query: str, contexts: List[str]) -> str:
        """Generate answer using LLM with retrieved contexts"""
        context_text = "\n\n".join([f"Context {i+1}:\n{ctx}" for i, ctx in enumerate(contexts)])

        prompt = f"""Based on the following product information, please answer the user's question accurately and comprehensively.

Product Information:
{context_text}

Question: {query}

Instructions:
- Use only the information provided in the contexts
- Be specific and include relevant details (prices, ratings, quantities, etc.)
- If the information is not available in the contexts, clearly state that
- Format your response clearly and concisely

Answer:"""

        try:
            response = self.llm.predict(prompt)
            return response.strip()
        except Exception as e:
            return f"Error generating response: {str(e)}"

    def query(self, question: str) -> Dict:
        """Complete RAG pipeline: retrieve + generate"""
        start_time = time.time()

        # Retrieve relevant contexts
        contexts = self.retrieve(question)

        # Generate answer
        answer = self.generate_answer(question, contexts)

        end_time = time.time()

        return {
            'question': question,
            'answer': answer,
            'contexts': contexts,
            'response_time': end_time - start_time
        }

# Initialize RAG system
print("Initializing RAG system...")
rag_system = RAGSystem(config, vector_db)
print("RAG system ready!")

In [ ]:
# ===============================================
# SECTION 7: TESTING THE RAG SYSTEM
# ===============================================

# **Text:** Testing the RAG system with sample queries.

# **Paragraph:** This section provides a set of test questions to evaluate the performance of the RAG system. It iterates through these questions, queries the system, and prints the generated answer and the response time for each question. This helps in quickly verifying if the system is working as expected.

# Test queries
test_questions = [
    "What is the most expensive product?",
    "Which products have the highest rating?",
    "What electronics are currently in stock?",
    "Which category has the best customer satisfaction?",
    "What products have discounts available?",
    "Show me products with low stock levels",
    "What is the average price of clothing items?",
    "Which products generated the most revenue?"
]

print("Testing RAG system with sample questions...\n")

test_results = []
for question in test_questions[:4]:  # Test first 4 questions
    print(f"Question: {question}")
    result = rag_system.query(question)
    print(f"Answer: {result['answer']}")
    print(f"Response time: {result['response_time']:.2f} seconds")
    print("-" * 80)
    test_results.append(result)

In [ ]:
# ===============================================
# SECTION 8: EVALUATION DATASET GENERATION
# ===============================================

# **Text:** Generating an evaluation dataset for RAGAS.

# **Paragraph:** This section defines a function `generate_evaluation_dataset` to create a dataset specifically for evaluating the RAG system using the RAGAS framework. It generates questions based on templates and extracts the corresponding ground truth answers from the original DataFrame. This dataset will be used to assess the quality of the RAG system's responses.

def generate_evaluation_dataset(df: pd.DataFrame, num_questions: int = 30) -> List[Dict]:
    """Generate evaluation dataset for RAGAS"""
    import random

    evaluation_data = []

    # Question templates with expected answers
    templates = [
        {
            "template": "What is the price of {product}?",
            "type": "price_query",
            "answer_func": lambda row: f"The price of {row['product_name']} is ${row['price']}."
        },
        {
            "template": "What is the rating of {product}?",
            "type": "rating_query",
            "answer_func": lambda row: f"The rating of {row['product_name']} is {row['rating']} out of 5."
        },
        {
            "template": "How many units of {product} were sold?",
            "type": "sales_query",
            "answer_func": lambda row: f"{row['product_name']} sold {row['sales_quantity']} units."
        },
        {
            "template": "What category does {product} belong to?",
            "type": "category_query",
            "answer_func": lambda row: f"{row['product_name']} belongs to the {row['category']} category."
        }
    ]

    # Generate questions
    products = df['product_name'].unique()

    for i in range(num_questions):
        template_info = random.choice(templates)
        product = random.choice(products)

        question = template_info["template"].format(product=product)

        # Get product info
        product_row = df[df['product_name'] == product].iloc[0]
        ground_truth = template_info["answer_func"](product_row)

        # Get relevant context
        context = f"Product: {product_row['product_name']}, Price: ${product_row['price']}, "
        context += f"Rating: {product_row['rating']}, Sales: {product_row['sales_quantity']}, "
        context += f"Category: {product_row['category']}"

        evaluation_data.append({
            'question': question,
            'ground_truth': ground_truth,
            'contexts': [context],
            'answer': ground_truth  # Will be replaced with RAG answer during evaluation
        })

    return evaluation_data

# Generate evaluation dataset
print("Generating evaluation dataset...")
eval_data = generate_evaluation_dataset(df, num_questions=20)
print(f"Generated {len(eval_data)} evaluation questions")

# Show sample evaluation data
print("\nSample evaluation questions:")
for i, item in enumerate(eval_data[:3]):
    print(f"{i+1}. Question: {item['question']}")
    print(f"    Ground Truth: {item['ground_truth']}")
    print()

In [ ]:
# ===============================================
# SECTION 9: RAG EVALUATION WITH RAGAS
# ===============================================

# **Text:** Evaluating the RAG system using RAGAS.

# **Paragraph:** This section defines a function `evaluate_rag_system` that takes the RAG system and the evaluation dataset as input and uses the RAGAS framework to assess its performance. It generates answers for the evaluation questions using the RAG system and then calculates several metrics such as context precision, context recall, faithfulness, answer relevancy, and answer correctness.

def evaluate_rag_system(rag_system: RAGSystem, eval_data: List[Dict]) -> Dict:
    """Evaluate RAG system using RAGAS metrics"""

    # Generate answers using RAG system
    print("Generating answers for evaluation...")
    for item in eval_data:
        rag_result = rag_system.query(item['question'])
        item['answer'] = rag_result['answer']
        item['contexts'] = rag_result['contexts']

    # Convert to RAGAS format
    dataset_dict = {
        'question': [item['question'] for item in eval_data],
        'answer': [item['answer'] for item in eval_data],
        'contexts': [item['contexts'] for item in eval_data],
        'ground_truth': [item['ground_truth'] for item in eval_data]
    }

    dataset = Dataset.from_dict(dataset_dict)

    # Run evaluation
    print("Running RAGAS evaluation...")
    try:
        result = evaluate(
            dataset,
            metrics=[
                context_precision,
                context_recall,
                faithfulness,
                answer_relevancy,
                answer_correctness
            ],
        )
        return result
    except Exception as e:
        print(f"Evaluation error: {str(e)}")
        return None

# Note: Uncomment below to run evaluation (requires valid OpenAI API key)
eval_results = evaluate_rag_system(rag_system, eval_data[:10])  # Evaluate first 10 questions
if eval_results:
     print("Evaluation Results:")
     for metric, score in eval_results.items():
         print(f"{metric}: {score:.4f}")

In [ ]:
# ===============================================
# SECTION 10: VISUALIZATION AND ANALYTICS
# ===============================================

# **Text:** Creating a visualization dashboard for the RAG system.

# **Paragraph:** This section defines a function `create_rag_analytics_dashboard` that generates an interactive dashboard using Plotly. The dashboard provides an overview of the e-commerce dataset through various plots such as product distribution by category, price distribution, rating distribution, and revenue by category. It also visualizes the response times of the RAG system for the test queries.

def create_rag_analytics_dashboard(df: pd.DataFrame, test_results: List[Dict]):
    """Create analytics dashboard for RAG system performance"""

    # Dataset overview
    fig1 = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Products by Category', 'Price Distribution',
                        'Rating Distribution', 'Revenue by Category'),
        specs=[[{'type': 'domain'}, {'type': 'histogram'}],
               [{'type': 'histogram'}, {'type': 'bar'}]]
    )

    # Products by category (pie chart)
    category_counts = df['category'].value_counts()
    fig1.add_trace(
        go.Pie(labels=category_counts.index, values=category_counts.values, name="Products"),
        row=1, col=1
    )

    # Price distribution
    fig1.add_trace(
        go.Histogram(x=df['price'], name="Price", nbinsx=30),
        row=1, col=2
    )

    # Rating distribution
    fig1.add_trace(
        go.Histogram(x=df['rating'], name="Rating", nbinsx=20),
        row=2, col=1
    )

    # Revenue by category
    revenue_by_category = df.groupby('category')['revenue'].sum().sort_values(ascending=True)
    fig1.add_trace(
        go.Bar(x=revenue_by_category.values, y=revenue_by_category.index,
                orientation='h', name="Revenue"),
        row=2, col=2
    )

    fig1.update_layout(height=800, title_text="E-commerce Dataset Overview")
    fig1.show()

    # RAG Performance metrics
    if test_results:
        response_times = [result['response_time'] for result in test_results]
        questions = [result['question'][:50] + "..." if len(result['question']) > 50
                     else result['question'] for result in test_results]

        fig2 = go.Figure()
        fig2.add_trace(go.Bar(
            x=questions,
            y=response_times,
            name='Response Time (seconds)',
            text=[f"{t:.2f}s" for t in response_times],
            textposition='auto'
        ))

        fig2.update_layout(
            title="RAG System Response Times",
            xaxis_title="Questions",
            yaxis_title="Response Time (seconds)",
            xaxis_tickangle=-45,
            height=500
        )
        fig2.show()

# Create dashboard
print("Creating analytics dashboard...")
create_rag_analytics_dashboard(df, test_results)

In [ ]:
# ===============================================
# SECTION 11: ADVANCED RAG FEATURES
# ===============================================

# **Text:** Implementing advanced features for the RAG system.

# **Paragraph:** This section extends the basic `RAGSystem` with an `AdvancedRAGSystem` class. It introduces features like querying with metadata filters (e.g., by category or price range) and tracking query history to provide basic analytics on the system's usage. Note that the filtering is currently implemented at the application level and would ideally be handled by the vector database for efficiency.

class AdvancedRAGSystem(RAGSystem):
    """Enhanced RAG system with additional features"""

    def __init__(self, config: RAGConfig, vector_db: VectorDatabase):
        super().__init__(config, vector_db)
        self.query_history = []

    def query_with_filter(self, question: str, category_filter: str = None,
                                price_range: tuple = None) -> Dict:
        """Query with metadata filtering"""
        # This would require enhanced vector DB implementation
        # For now, we'll demonstrate the concept
        basic_result = self.query(question)

        if category_filter or price_range:
            # Filter contexts based on metadata
            filtered_contexts = []
            for context in basic_result['contexts']:
                # Simple filtering logic (in practice, this should be done at vector DB level)
                if category_filter and category_filter.lower() in context.lower():
                    filtered_contexts.append(context)
                elif not category_filter:
                    filtered_contexts.append(context)

            if filtered_contexts:
                basic_result['contexts'] = filtered_contexts
                basic_result['answer'] = self.generate_answer(question, filtered_contexts)

        self.query_history.append(basic_result)
        return basic_result

    def get_query_analytics(self) -> Dict:
        """Get analytics on query history"""
        if not self.query_history:
            return {}

        total_queries = len(self.query_history)
        avg_response_time = np.mean([q['response_time'] for q in self.query_history])

        return {
            'total_queries': total_queries,
            'average_response_time': avg_response_time,
            'recent_queries': [q['question'] for q in self.query_history[-5:]]
        }

# Initialize advanced RAG system
advanced_rag = AdvancedRAGSystem(config, vector_db)

# Test advanced features
print("Testing advanced RAG features...")
filtered_result = advanced_rag.query_with_filter(
    "What are the best electronics?",
    category_filter="Electronics"
)
print(f"Filtered query result: {filtered_result['answer'][:200]}...")

In [ ]:
# ===============================================
# SECTION 12: INTERACTIVE DEMO
# ===============================================

# **Text:** Running an interactive demo of the RAG system.

# **Paragraph:** This section provides an interactive command-line interface to test the RAG system with custom questions. Users can type in their questions about the e-commerce dataset, and the system will retrieve relevant information and generate an answer. The demo continues until the user types 'quit'. It also offers an option to display the retrieved contexts.

def interactive_rag_demo():
    """Interactive demo function for testing RAG system"""
    print("\n" + "="*60)
    print("INTERACTIVE RAG SYSTEM DEMO")
    print("="*60)
    print("You can ask questions about the e-commerce dataset.")
    print("Example questions:")
    print("- What is the most expensive product?")
    print("- Which electronics have high ratings?")
    print("- Show me products with discounts")
    print("- What's the average price in each category?")
    print("\nType 'quit' to exit the demo.")
    print("-"*60)

    while True:
        user_question = input("\nYour question: ").strip()

        if user_question.lower() in ['quit', 'exit', 'q']:
            print("Demo ended. Thank you!")
            break

        if not user_question:
            print("Please enter a question.")
            continue

        print("\nProcessing your question...")
        result = rag_system.query(user_question)

        print(f"\nAnswer: {result['answer']}")
        print(f"Response time: {result['response_time']:.2f} seconds")

        # Show relevant contexts (optional)
        show_contexts = input("\nShow relevant contexts? (y/n): ").strip().lower()
        if show_contexts == 'y':
            print("\nRelevant contexts:")
            for i, context in enumerate(result['contexts'][:2], 1):
                print(f"\nContext {i}:")
                print(context[:300] + "..." if len(context) > 300 else context)

# Uncomment to run interactive demo
interactive_rag_demo()

In [ ]:
# ===============================================
# SECTION 13: SAVING AND LOADING MODELS
# ===============================================

# **Text:** Saving and loading the RAG system components.

# **Paragraph:** This section implements functionality to save the trained RAG system (specifically the metadata and implicitly the persisted vector database) to a specified path. It also includes a function to load the saved RAG system components, allowing for the system to be reused without retraining and rebuilding the vector database.

def save_rag_system(rag_system: RAGSystem, vector_db: VectorDatabase, save_path: str):
    """Save RAG system components"""
    import pickle

    # Save metadata
    metadata = {
        'config': config.__dict__,
        'num_documents': len(chunks),
        'embedding_model': config.embedding_model,
        'created_at': datetime.now().isoformat()
    }

    with open(f"{save_path}/metadata.json", 'w') as f:
        json.dump(metadata, f, indent=2)

    # Vector database is already persisted
    print(f"RAG system saved to {save_path}")

def load_rag_system(save_path: str) -> tuple:
    """Load RAG system components"""
    # Load metadata
    with open(f"{save_path}/metadata.json", 'r') as f:
        metadata = json.load(f)

    # Recreate components
    loaded_config = RAGConfig()
    loaded_vector_db = VectorDatabase(loaded_config)
    loaded_rag_system = RAGSystem(loaded_config, loaded_vector_db)

    print(f"RAG system loaded from {save_path}")
    return loaded_rag_system, loaded_vector_db, metadata

# Save the current system
save_path = "/content/rag_system"
os.makedirs(save_path, exist_ok=True)
save_rag_system(rag_system, vector_db, save_path)

In [ ]:
# ===============================================
# SECTION 14: SUMMARY AND NEXT STEPS
# ===============================================

print("\n" + "="*80)
print("RAG SYSTEM SETUP COMPLETE!")
print("="*80)
print(f"✅ Generated dataset with {len(df)} products")
print(f"✅ Created {len(chunks)} text chunks")
print(f"✅ Built vector database with embeddings")
print(f"✅ Implemented complete RAG pipeline")
print(f"✅ Tested system with sample queries")
print(f"✅ Created evaluation framework")
print(f"✅ Generated analytics dashboard")

print("\nSystem Components:")
print(f"- Embedding Model: {config.embedding_model}")
print(f"- Vector Database: ChromaDB")
print(f"- LLM: GPT-3.5-turbo")
print(f"- Top-K Retrieval: {config.top_k_retrieval}")

print("\nNext Steps:")
print("1. Set your OpenAI API key in the config")
print("2. Run the interactive demo to test queries")
print("3. Evaluate system performance with RAGAS")
print("4. Customize for your specific dataset")
print("5. Deploy to production environment")

print("\nFiles Created:")
print("- /content/ecommerce_products.csv")
print("- /content/chroma_db/ (vector database)")
print("- /content/rag_system/ (saved model)")

print("\n" + "="*80)
